In [1]:
import pandas as pd
import pandasql

from pandasql import sqldf 
sql_run = lambda q: sqldf(q, globals())

In [2]:
worldcity = pd.read_csv('https://raw.githubusercontent.com/vertikalwil/Data-Analyst/main/worldcities.csv', sep = ';', encoding = 'latin-1')
uscity = pd.read_csv('https://raw.githubusercontent.com/vertikalwil/Data-Analyst/main/uscities.csv', sep = ';', encoding = 'latin-1')
uscity.columns = ['city', 'state_id', 'state_name', 'county_fips', 'county_name',
       'lat', 'lng', 'population', 'density', 'source', 'military',
       'incorporated', 'timezone', 'ranking', 'zips', 'id']

### Tutorial Bagaimana Cara Melakukan Query Menggunakan Pandasql

Contoh: Bagaimana melakukan query untuk mendapatkan kota-kota dari worldcity yang merupakan:
- berada di antara 33 - 40 'latitude'
- berada di antara 100 - 140 'longitude'
- merupakan kota 'primary' pada kolom 'capital'
- memiliki populasi lebih dari 20 juta

In [3]:
worldcity[(worldcity.country == 'Philippines') & (worldcity.city == 'San Jose')]

,city,lat,lng,country,iso2,iso3,capital,population,id
3672,San Jose,12.3528,121.0676,Philippines,PH,PHL,NaN,143430.0,1608355709
3747,San Jose,15.7833,121.0000,Philippines,PH,PHL,NaN,139738.0,1608000478
5600,San Jose,10.7500,121.9500,Philippines,PH,PHL,admin,62534.0,1608138574
10014,San Jose,10.1800,125.5683,Philippines,PH,PHL,admin,NaN,1608435522
10985,San Jose,13.8772,121.1050,Philippines,PH,PHL,NaN,76971.0,1608353630
13936,San Jose,13.7000,123.5167,Philippines,PH,PHL,NaN,40623.0,1608136409
16693,San Jose,10.0083,125.5889,Philippines,PH,PHL,NaN,27487.0,1608706852
19751,San Jose,9.4167,123.2333,Philippines,PH,PHL,NaN,20413.0,1608586571
21240,San Jose,12.5310,124.4870,Philippines,PH,PHL,NaN,17561.0,1608912746


### Question 1 Point : 40

Temukan kota-kota dari worldcity yang memenuhi semua kondisi ini:
- populasi kota tersebut melebih populasi dari kota terbanyak penduduknya di negara Filipina. (Maksudnya: misalnya kota X adalah kota di Filipina yang paling banyak penduduknya. Kita mau daftar kota-kota yang jumlah penduduknya lebih besar dari kota X tersebut), DAN
- ada kota di Amerika Serikat (uscity) dengan nama kota yang sama dengan kota-kota pilihan

Keluarkan hanya nama kota-kota yang memenuhi 2 syarat di atas saja, tidak usah kolom-kolom lain.

##### NOTE: Hilangkan duplicate rows.

Lakukan hanya 1x query saja, jangan melakukan lebih dari 1 query

In [4]:
#There are many duplicates city in 'Phillipines', we need to sum all the city's populations first by grouping them.
worldcity[worldcity.country == 'Philippines'].city.duplicated().sum()

228

In [5]:
query = """
WITH tabel as 
(SELECT city as kota FROM worldcity
WHERE population > (SELECT MAX(jumlah) FROM (SELECT SUM(population) as jumlah FROM worldcity WHERE country == 'Philippines' GROUP BY city)))

SELECT kota from tabel
WHERE kota IN (SELECT city FROM uscity)
"""
sql_run(query)

,kota
0,Delhi


In [19]:
uscity[uscity.city == 'Franklin']

,city,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
557,Franklin,TN,Tennessee,47187,Williamson,35.9216,-86.8525,83097,771,polygon,False,True,America/Chicago,3,37069 37064 37067 37065 37068,1840013380
1369,Franklin,WI,Wisconsin,55079,Milwaukee,42.8854,-88.0104,35811,399,polygon,False,True,America/Chicago,3,53132,1840003043
1438,Franklin,MA,Massachusetts,25021,Norfolk,42.0862,-71.4113,34087,494,polygon,False,True,America/New_York,3,2038,1840031178
1478,Franklin,NJ,New Jersey,34037,Sussex,41.1101,-74.5886,33153,418,polygon,False,True,America/New_York,3,7416,1840003480
1877,Franklin,IN,Indiana,18081,Johnson,39.4938,-86.0546,25608,735,polygon,False,True,America/Indiana/Indianapolis,3,46131,1840008460
3453,Franklin,LA,Louisiana,22101,St. Mary,29.7851,-91.5098,12803,261,polygon,False,True,America/Chicago,3,70538,1840014019
3731,Franklin,OH,Ohio,39165,Warren,39.5538,-84.2950,11612,488,polygon,False,True,America/New_York,3,45005,1840008466
4129,Franklin,KY,Kentucky,21213,Simpson,36.7178,-86.5595,10118,238,polygon,False,True,America/Chicago,3,42134 42135,1840013290
4213,Franklin,NH,New Hampshire,33013,Merrimack,43.4499,-71.6691,9817,122,polygon,False,True,America/New_York,3,3235,1840002748
4570,Franklin,PA,Pennsylvania,42121,Venango,41.3936,-79.8425,8802,496,polygon,False,True,America/New_York,3,16323,1840003425


In [16]:
uscity[uscity.city == 'Kansas City']

,city,state_id,state_name,county_fips,county_name,lat,lng,population,density,source,military,incorporated,timezone,ranking,zips,id
33,Kansas City,MO,Missouri,29095,Jackson,39.1239,-94.5541,1636715,607,polygon,False,True,America/Chicago,1,64161 64163 64165 64167 64053 64151 64153 6415...,1840008535
291,Kansas City,KS,Kansas,20209,Wyandotte,39.1234,-94.7443,152960,473,polygon,False,True,America/Chicago,2,66101 66102 66103 66104 66105 66106 66109 6611...,1840001626


In [11]:
#check using python pandas to make sure.
df = worldcity[worldcity.population > worldcity[worldcity.country == 'Philippines'].groupby('city')['population'].agg('mean').max()]
df[df.city.isin(uscity.city)].city

2    Delhi
Name: city, dtype: object

### Question 2 Point : 60

Temukan kota-kota di Amerika Serikat (uscity) yang memenuhi semua kondisi berikut:
- memiliki nama 'city' yang tidak sama dengan 'state_name', dan juga
- memiliki populasi di atas rata-rata populasi kota-kota di 'county_name' Miami-Dade, dan juga
- terdiri dari dua atau lebih kata (contoh: Los Angeles, New York)

Output yang diharapkan hanya kolom 'city' saja, tidak usah kolom yang lain

Lakukan hanya 1x query saja, jangan melakukan lebih dari 1 query

In [7]:
#no duplicate city in 'Miami-Dade', Here we can use avg population directly without grouping the city first.
uscity[uscity.county_name == 'Miami-Dade'].city.duplicated().sum()

0

In [8]:
query = """
WITH tabel as 
(SELECT *, LENGTH(city) - LENGTH(replace(city,' ','')) as koma FROM uscity)
SELECT city FROM tabel
WHERE city != state_name AND
population > (SELECT AVG(population) FROM uscity WHERE county_name = 'Miami-Dade') AND
koma > 0
"""
soal2df = sql_run(query)
soal2df

,city
0,Los Angeles
1,San Francisco
2,San Diego
3,Las Vegas
4,San Antonio
...,...
85,South Lyon
86,Lehigh Acres
87,East Los Angeles
88,St. George


In [9]:
#check using python pandas to make sure.
df = uscity[uscity.city != uscity.state_name]
df = df[df.population > uscity[uscity.county_name == 'Miami-Dade'].population.mean()]
df[df.city.str.count(" ") > 0][['city']].reset_index(drop=True)

,city
0,Los Angeles
1,San Francisco
2,San Diego
3,Las Vegas
4,San Antonio
...,...
85,South Lyon
86,Lehigh Acres
87,East Los Angeles
88,St. George


In [10]:
df[df.city.str.count(" ") > 0][['city']].reset_index(drop=True).equals(soal2df)

True